In [1]:
import tensorflow as tf
import numpy as np
import cv2
from keras.utils import Sequence
from keras.applications.mobilenet_v3 import preprocess_input
from glob import glob
import os
from sklearn.model_selection import train_test_split

from keras.applications import MobileNetV3Large
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.optimizers import AdamW
from keras.losses import CategoricalCrossentropy
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.metrics import TopKCategoricalAccuracy
from keras.layers import RandomRotation
from keras.callbacks import LearningRateScheduler



import math
import json


import re
print(tf.__version__)
import tensorflow_probability as tfp
tfd = tfp.distributions


2.15.0




In [2]:
# Load class names
with open("food-101/meta/classes.txt", "r") as file:
    classes = [line.strip() for line in file]

# Read image paths
with open("food-101/meta/train.txt", "r") as file:
    train_paths = [line.strip() for line in file]

with open("food-101/meta/test.txt", "r") as file:
    test_paths = [line.strip() for line in file]

# Helper function to convert label format
def to_snake_case(name):
    return name.replace(" ", "_").lower()

# Convert paths to full paths
train_full_paths = [os.path.join("food-101/images", p + ".jpg") for p in train_paths]
train_labels = [to_snake_case(p.split("/")[0]) for p in train_paths]

test_full_paths = [os.path.join("food-101/images", p + ".jpg") for p in test_paths]
test_labels = [to_snake_case(p.split("/")[0]) for p in test_paths]

# Map labels to indices
class_to_index = {cls: idx for idx, cls in enumerate(classes)}
train_label_indices = [class_to_index[label] for label in train_labels]
test_label_indices = [class_to_index[label] for label in test_labels]

# ✂️ Split 80% train, 20% val
train_paths_split, val_paths_split, train_labels_split, val_labels_split = train_test_split(
    train_full_paths, train_label_indices, test_size=0.2, stratify=train_label_indices, random_state=42
)

# Constants
IMG_SIZE = 224
BATCH_SIZE = 64
NUM_CLASSES = len(classes)

rotation_layer = RandomRotation(factor=0.014)

# Augmentation function
def augment_image(img):
    img = rotation_layer(img) 
    img = tf.image.resize_with_crop_or_pad(img, 256, 256)
    img = tf.image.random_crop(img, size=[224, 224, 3])
    img = tf.image.random_flip_left_right(img)

    img = tf.image.random_brightness(img, max_delta=0.1)         
    img = tf.image.random_contrast(img, lower=0.9, upper=1.1)    
    img = tf.image.random_saturation(img, lower=0.95, upper=1.05) 
    img = tf.image.random_hue(img, max_delta=0.05)                

    img = tf.clip_by_value(img, 0.0, 255.0)
    return img
# Preprocess each sample
def process_example(path, label, augment=False):
    img_raw = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img_raw, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    if augment:
        img = augment_image(img)
    img = preprocess_input(img)
    label = tf.one_hot(label, NUM_CLASSES)
    return img, label

# Dataset creation function
def create_dataset(paths, labels, augment=False, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(paths))
    ds = ds.map(lambda x, y: process_example(x, y, augment), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

val_ds = create_dataset(val_paths_split, val_labels_split, augment=False, shuffle=False)
test_ds = create_dataset(test_full_paths, test_label_indices, shuffle=False)

In [3]:
def get_cosine_annealing_with_restarts(
    initial_lr=1e-3,
    min_lr=1e-7,
    warmup_epochs=5,
    T_0=20,
    T_mult=1
):
    def scheduler(epoch, lr):

        # Warmup phase
        if epoch < warmup_epochs:
            return float(initial_lr * (epoch + 1) / warmup_epochs)

        # Cosine annealing with restarts
        t = epoch - warmup_epochs  # Epochs after warmup
        T_i = T_0
        cumulative = 0

        while t >= T_i:
            t -= T_i
            cumulative += T_i
            T_i *= T_mult

        progress = t / T_i
        cosine_decay = 0.5 * (1 + math.cos(math.pi * progress))
        return float(min_lr + (initial_lr - min_lr) * cosine_decay)

    return scheduler

In [4]:
# No Augmentation
train_ds = create_dataset(train_paths_split, train_labels_split, augment=True)

loss_fn = CategoricalCrossentropy(label_smoothing=0.05)


base_model = MobileNetV3Large(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(128, activation='relu')(x)
output = Dense(101, activation='softmax')(x)

checkpoint_cb = ModelCheckpoint(
    filepath='D:\\Skripsi\\MobilNetV3-Food101-Rev-Baseline\\Baseline_Model_MobileNetV3-1-BasicAugment.keras',          
    monitor='val_accuracy',            
    mode='max',                         
    save_best_only=True,                            
    verbose=1
)

lr_scheduler = LearningRateScheduler(get_cosine_annealing_with_restarts())

model = Model(inputs=base_model.input, outputs=output)
model.compile(
    optimizer=AdamW(learning_rate=1e-3, weight_decay=1e-5), 
    loss=loss_fn, 
    metrics=['accuracy', TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
)

history = model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=5, 
    callbacks=[checkpoint_cb, lr_scheduler]
)

total_layers = len(base_model.layers)

unfreeze_count = int(total_layers * 0.3)

for layer in base_model.layers[-unfreeze_count:]:
    layer.trainable = True

model.compile(
    optimizer=AdamW(learning_rate=1e-3, weight_decay=1e-5), 
    loss=loss_fn, 
    metrics=['accuracy', TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
)

history_fine_tune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,
    initial_epoch=5,
    callbacks=[checkpoint_cb, lr_scheduler]
)

combined_history = {}
for key in history.history.keys():
    combined_history[key] = history.history[key] + history_fine_tune.history[key]

with open('skripsi/history/Model_History_MobilNetV3-1-BasicAugment.json', 'w') as f:
    json.dump(combined_history, f)


Epoch 1/5

947/947 [==============================] - ETA: 0s - loss: 3.3515 - accuracy: 0.2608 - top_5_accuracy: 0.4938
Epoch 1: val_accuracy improved from -inf to 0.49168, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-1-BasicAugment.keras
947/947 [==============================] - 746s 785ms/step - loss: 3.3515 - accuracy: 0.2608 - top_5_accuracy: 0.4938 - val_loss: 2.3189 - val_accuracy: 0.4917 - val_top_5_accuracy: 0.7686 - lr: 2.0000e-04
Epoch 2/5
947/947 [==============================] - ETA: 0s - loss: 2.4219 - accuracy: 0.4599 - top_5_accuracy: 0.7389
Epoch 2: val_accuracy improved from 0.49168 to 0.56436, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-1-BasicAugment.keras
947/947 [==============================] - 731s 771ms/step - loss: 2.4219 - accuracy: 0.4599 - top_5_accuracy: 0.7389 - val_loss: 2.0140 - val_accuracy: 0.5644 - val_top_5_accuracy: 0.8226 - lr: 4.0000e-04
Epoch 3/5
947/947 [====

TypeError: Object of type float32 is not JSON serializable

Epoch 1/5
947/947 [==============================] - ETA: 0s - loss: 3.1803 - accuracy: 0.2992 - top_5_accuracy: 0.5402
Epoch 1: val_accuracy improved from -inf to 0.51122, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 722s 758ms/step - loss: 3.1803 - accuracy: 0.2992 - top_5_accuracy: 0.5402 - val_loss: 2.2619 - val_accuracy: 0.5112 - val_top_5_accuracy: 0.7741 - lr: 2.0000e-04
Epoch 2/5
947/947 [==============================] - ETA: 0s - loss: 2.2572 - accuracy: 0.5024 - top_5_accuracy: 0.7753
Epoch 2: val_accuracy improved from 0.51122 to 0.57710, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 665s 702ms/step - loss: 2.2572 - accuracy: 0.5024 - top_5_accuracy: 0.7753 - val_loss: 1.9655 - val_accuracy: 0.5771 - val_top_5_accuracy: 0.8299 - lr: 4.0000e-04
Epoch 3/5
947/947 [==============================] - ETA: 0s - loss: 2.0486 - accuracy: 0.5543 - top_5_accuracy: 0.8165
Epoch 3: val_accuracy improved from 0.57710 to 0.59960, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 666s 704ms/step - loss: 2.0486 - accuracy: 0.5543 - top_5_accuracy: 0.8165 - val_loss: 1.8882 - val_accuracy: 0.5996 - val_top_5_accuracy: 0.8446 - lr: 6.0000e-04
Epoch 4/5
947/947 [==============================] - ETA: 0s - loss: 1.9556 - accuracy: 0.5801 - top_5_accuracy: 0.8326
Epoch 4: val_accuracy improved from 0.59960 to 0.60587, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 666s 703ms/step - loss: 1.9556 - accuracy: 0.5801 - top_5_accuracy: 0.8326 - val_loss: 1.8633 - val_accuracy: 0.6059 - val_top_5_accuracy: 0.8471 - lr: 8.0000e-04
Epoch 5/5
947/947 [==============================] - ETA: 0s - loss: 1.8980 - accuracy: 0.5956 - top_5_accuracy: 0.8424
Epoch 5: val_accuracy improved from 0.60587 to 0.61069, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 665s 702ms/step - loss: 1.8980 - accuracy: 0.5956 - top_5_accuracy: 0.8424 - val_loss: 1.8535 - val_accuracy: 0.6107 - val_top_5_accuracy: 0.8461 - lr: 0.0010
Epoch 6/25
947/947 [==============================] - ETA: 0s - loss: 1.7989 - accuracy: 0.6256 - top_5_accuracy: 0.8614
Epoch 6: val_accuracy did not improve from 0.61069
947/947 [==============================] - 907s 954ms/step - loss: 1.7989 - accuracy: 0.6256 - top_5_accuracy: 0.8614 - val_loss: 2.9159 - val_accuracy: 0.4632 - val_top_5_accuracy: 0.7349 - lr: 0.0010
Epoch 7/25
947/947 [==============================] - ETA: 0s - loss: 1.4213 - accuracy: 0.7300 - top_5_accuracy: 0.9203
Epoch 7: val_accuracy did not improve from 0.61069
947/947 [==============================] - 903s 953ms/step - loss: 1.4213 - accuracy: 0.7300 - top_5_accuracy: 0.9203 - val_loss: 2.0250 - val_accuracy: 0.5919 - val_top_5_accuracy: 0.8366 - lr: 9.9384e-04
Epoch 8/25
947/947 [==============================] - ETA: 0s - loss: 1.2321 - accuracy: 0.7841 - top_5_accuracy: 0.9435
Epoch 8: val_accuracy improved from 0.61069 to 0.67281, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 905s 956ms/step - loss: 1.2321 - accuracy: 0.7841 - top_5_accuracy: 0.9435 - val_loss: 1.6919 - val_accuracy: 0.6728 - val_top_5_accuracy: 0.8787 - lr: 9.7553e-04
Epoch 9/25
947/947 [==============================] - ETA: 0s - loss: 1.0752 - accuracy: 0.8306 - top_5_accuracy: 0.9628
Epoch 9: val_accuracy did not improve from 0.67281
947/947 [==============================] - 902s 953ms/step - loss: 1.0752 - accuracy: 0.8306 - top_5_accuracy: 0.9628 - val_loss: 1.7711 - val_accuracy: 0.6686 - val_top_5_accuracy: 0.8720 - lr: 9.4551e-04
Epoch 10/25
947/947 [==============================] - ETA: 0s - loss: 0.9441 - accuracy: 0.8704 - top_5_accuracy: 0.9769
Epoch 10: val_accuracy improved from 0.67281 to 0.68812, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 904s 955ms/step - loss: 0.9441 - accuracy: 0.8704 - top_5_accuracy: 0.9769 - val_loss: 1.6486 - val_accuracy: 0.6881 - val_top_5_accuracy: 0.8878 - lr: 9.0452e-04
Epoch 11/25
947/947 [==============================] - ETA: 0s - loss: 0.8378 - accuracy: 0.9019 - top_5_accuracy: 0.9865
Epoch 11: val_accuracy did not improve from 0.68812
947/947 [==============================] - 903s 953ms/step - loss: 0.8378 - accuracy: 0.9019 - top_5_accuracy: 0.9865 - val_loss: 1.6471 - val_accuracy: 0.6876 - val_top_5_accuracy: 0.8826 - lr: 8.5357e-04
Epoch 12/25
947/947 [==============================] - ETA: 0s - loss: 0.7490 - accuracy: 0.9308 - top_5_accuracy: 0.9927
Epoch 12: val_accuracy improved from 0.68812 to 0.70871, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 904s 955ms/step - loss: 0.7490 - accuracy: 0.9308 - top_5_accuracy: 0.9927 - val_loss: 1.5814 - val_accuracy: 0.7087 - val_top_5_accuracy: 0.8935 - lr: 7.9391e-04
Epoch 13/25
947/947 [==============================] - ETA: 0s - loss: 0.6760 - accuracy: 0.9537 - top_5_accuracy: 0.9965
Epoch 13: val_accuracy improved from 0.70871 to 0.72271, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 904s 955ms/step - loss: 0.6760 - accuracy: 0.9537 - top_5_accuracy: 0.9965 - val_loss: 1.5325 - val_accuracy: 0.7227 - val_top_5_accuracy: 0.9007 - lr: 7.2702e-04
Epoch 14/25
947/947 [==============================] - ETA: 0s - loss: 0.6212 - accuracy: 0.9702 - top_5_accuracy: 0.9982
Epoch 14: val_accuracy improved from 0.72271 to 0.73201, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 926s 978ms/step - loss: 0.6212 - accuracy: 0.9702 - top_5_accuracy: 0.9982 - val_loss: 1.4986 - val_accuracy: 0.7320 - val_top_5_accuracy: 0.8966 - lr: 6.5454e-04
Epoch 15/25
947/947 [==============================] - ETA: 0s - loss: 0.5828 - accuracy: 0.9808 - top_5_accuracy: 0.9994
Epoch 15: val_accuracy did not improve from 0.73201
947/947 [==============================] - 923s 974ms/step - loss: 0.5828 - accuracy: 0.9808 - top_5_accuracy: 0.9994 - val_loss: 1.4756 - val_accuracy: 0.7310 - val_top_5_accuracy: 0.9017 - lr: 5.7826e-04
Epoch 16/25
947/947 [==============================] - ETA: 0s - loss: 0.5474 - accuracy: 0.9887 - top_5_accuracy: 0.9997
Epoch 16: val_accuracy improved from 0.73201 to 0.73828, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 904s 954ms/step - loss: 0.5474 - accuracy: 0.9887 - top_5_accuracy: 0.9997 - val_loss: 1.4551 - val_accuracy: 0.7383 - val_top_5_accuracy: 0.9073 - lr: 5.0005e-04
Epoch 17/25
947/947 [==============================] - ETA: 0s - loss: 0.5203 - accuracy: 0.9941 - top_5_accuracy: 0.9999
Epoch 17: val_accuracy improved from 0.73828 to 0.74310, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 903s 954ms/step - loss: 0.5203 - accuracy: 0.9941 - top_5_accuracy: 0.9999 - val_loss: 1.4333 - val_accuracy: 0.7431 - val_top_5_accuracy: 0.9051 - lr: 4.2184e-04
Epoch 18/25
947/947 [==============================] - ETA: 0s - loss: 0.5015 - accuracy: 0.9967 - top_5_accuracy: 1.0000
Epoch 18: val_accuracy improved from 0.74310 to 0.74832, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 906s 956ms/step - loss: 0.5015 - accuracy: 0.9967 - top_5_accuracy: 1.0000 - val_loss: 1.4157 - val_accuracy: 0.7483 - val_top_5_accuracy: 0.9070 - lr: 3.4556e-04
Epoch 19/25
947/947 [==============================] - ETA: 0s - loss: 0.4889 - accuracy: 0.9985 - top_5_accuracy: 1.0000
Epoch 19: val_accuracy improved from 0.74832 to 0.75234, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 905s 955ms/step - loss: 0.4889 - accuracy: 0.9985 - top_5_accuracy: 1.0000 - val_loss: 1.3981 - val_accuracy: 0.7523 - val_top_5_accuracy: 0.9098 - lr: 2.7308e-04
Epoch 20/25
947/947 [==============================] - ETA: 0s - loss: 0.4788 - accuracy: 0.9990 - top_5_accuracy: 1.0000
Epoch 20: val_accuracy improved from 0.75234 to 0.75360, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 904s 955ms/step - loss: 0.4788 - accuracy: 0.9990 - top_5_accuracy: 1.0000 - val_loss: 1.3929 - val_accuracy: 0.7536 - val_top_5_accuracy: 0.9087 - lr: 2.0619e-04
Epoch 21/25
947/947 [==============================] - ETA: 0s - loss: 0.4716 - accuracy: 0.9994 - top_5_accuracy: 1.0000
Epoch 21: val_accuracy improved from 0.75360 to 0.75399, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 904s 955ms/step - loss: 0.4716 - accuracy: 0.9994 - top_5_accuracy: 1.0000 - val_loss: 1.3875 - val_accuracy: 0.7540 - val_top_5_accuracy: 0.9112 - lr: 1.4653e-04
Epoch 22/25
947/947 [==============================] - ETA: 0s - loss: 0.4655 - accuracy: 0.9998 - top_5_accuracy: 1.0000
Epoch 22: val_accuracy improved from 0.75399 to 0.75723, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 904s 955ms/step - loss: 0.4655 - accuracy: 0.9998 - top_5_accuracy: 1.0000 - val_loss: 1.3824 - val_accuracy: 0.7572 - val_top_5_accuracy: 0.9123 - lr: 9.5582e-05
Epoch 23/25
947/947 [==============================] - ETA: 0s - loss: 0.4624 - accuracy: 0.9998 - top_5_accuracy: 1.0000
Epoch 23: val_accuracy improved from 0.75723 to 0.75762, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 904s 955ms/step - loss: 0.4624 - accuracy: 0.9998 - top_5_accuracy: 1.0000 - val_loss: 1.3826 - val_accuracy: 0.7576 - val_top_5_accuracy: 0.9117 - lr: 5.4591e-05
Epoch 24/25
947/947 [==============================] - ETA: 0s - loss: 0.4602 - accuracy: 0.9998 - top_5_accuracy: 1.0000
Epoch 24: val_accuracy improved from 0.75762 to 0.76040, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Baseline_Model_MobileNetV3-2.keras
947/947 [==============================] - 905s 955ms/step - loss: 0.4602 - accuracy: 0.9998 - top_5_accuracy: 1.0000 - val_loss: 1.3798 - val_accuracy: 0.7604 - val_top_5_accuracy: 0.9125 - lr: 2.4569e-05
Epoch 25/25
947/947 [==============================] - ETA: 0s - loss: 0.4593 - accuracy: 0.9999 - top_5_accuracy: 1.0000
Epoch 25: val_accuracy did not improve from 0.76040
947/947 [==============================] - 904s 955ms/step - loss: 0.4593 - accuracy: 0.9999 - top_5_accuracy: 1.0000 - val_loss: 1.3797 - val_accuracy: 0.7597 - val_top_5_accuracy: 0.9123 - lr: 6.2552e-06

In [5]:
results = model.evaluate(test_ds)

395/395 [==============================] - 240s 606ms/step - loss: 1.1386 - accuracy: 0.8094 - top_5_accuracy: 0.9449
